In [1]:
# -*- coding: utf-8 -*-
import sys
import os       
import pandas as pd
import datetime

os.chdir('D:/zxdf/Workspaces/PycharmProjects/flask_ml')
from app import config, Elasticsearch_Util
#from app.lib import nltk_text_analyze_lib as textAnalyzer
#from app.lib import sklearn_classification_lib as skclassification

starttime = datetime.datetime.now()
print ("当前的日期和时间是 %s" % starttime)

In [2]:
def getMoreDataFromES(self, index, doc_type,beginTime, endTime, source_include,size):
    querybody = {
        "query": {
            "bool": {
                "must": [
                    {
                        "range": {
                            "@timestamp": {
                                "from": beginTime,
                                "to": endTime
                            }
                        }
                    }
                ],
            }
        },
        "_source": {
            "includes": source_include,
            "excludes": []
        }
    }
    # Initialize the scroll
    print "Initialize the scroll"
    page = self.es.search(
        index=index,
        doc_type=doc_type,
        scroll='2m',
        size=size,
        body=querybody)
    sid = page['_scroll_id']
    scroll_size = page['hits']['total']
    
    print "sid: " + str(sid)
    print "scroll size: " + str(scroll_size)
    
    records = []
    for doc in page['hits']['hits']:
            source=doc['_source']
            source["id"]=doc['_id']
            #print source
            yield source
            records.append(source)
    
    df = None
   #Start scrolling
    while (scroll_size > 0):
        print "Scrolling..."
        page = self.es.scroll(scroll_id=sid, scroll='2m')
        # Update the scroll ID
        sid = page['_scroll_id']
        print "sid: " + str(sid)
        # Get the number of results that we returned in the last scroll
        scroll_size = len(page['hits']['hits'])
        
        for doc in page['hits']['hits']:
            source=doc['_source']
            source["id"]=doc['_id']
            
            #print source
            yield source
            records.append(source)
            
        print "scroll size: " + str(scroll_size)
        
    #if records:
    #    df = pd.DataFrame(records) 
    #    # Do something with the obtained page
    #return df

In [11]:
index = "1012-knowledge_training_normal-*"
es_type = "knowledge_training_normal"
beginTime = "2018-02-05T00:00:00.000+08:00"
endTime = "2018-02-06T00:00:00.000+08:00"
size = 10000
tag = "isException"
logmsg = "logBody"
source_include = [tag, logmsg]

es_util = Elasticsearch_Util()
train_data_normal = getMoreDataFromES(es_util, index=index, doc_type=es_type,
                           beginTime=beginTime, endTime=endTime,
                           size=size,
                           source_include=source_include)


In [12]:
for data in train_data_normal:
    print data

In [ ]:
import os
from flask import jsonify, request
from sklearn.externals import joblib

import nltk
import string
from nltk.corpus import stopwords

os.chdir('D:/zxdf/Workspaces/PycharmProjects/flask_ml')


# =============================================================================================================================
# normal_preprocessing02\teststack1618_804
model = joblib.load(r'app\rules\modelspkl\normal_preprocessing02\teststack1618_804\knowledge_cart.pkl')
tf_transformer = joblib.load(r'app\rules\modelspkl\normal_preprocessing02\teststack1618_804\knowledge_tf_transformer.pkl')
le = joblib.load(r'app\rules\modelspkl\normal_preprocessing02\teststack1618_804\knowledge_labelencoder.pkl')

def error_predict(testline):
    """
    获取用户信息
    :return: json
    """
    # auth = Auth()
    # result = auth.identify(request)

    # if (result['status']):
    # test_datas = readDataLine(testline)
    if len(testline) == 0:
        return "this is None"
    else:
        test_datas = []
        featureValue = testline.lower().strip()
        test_datas.append(featureValue)
        test_feature_datas = tf_transformer.transform(test_datas)
        # 进行预测
        pred = model.predict(test_feature_datas)
        result = le.inverse_transform(pred)
        print ('预测结果:%s,预测内容:%s' % (result[0], testline))
        
        #判断预测内容是否进行告警
        thresholdValueComparision()
        
        return result[0]

In [ ]:
for data in train_data_normal:
    error_predict(train_data_normal,logmsg)
    print data

In [ ]:
alertCommon(alertConfig):
    flagalert = false
    search_response = ""
            Long hitstotal = 0L;

            if (alertConfig.has_key("indexStatus") and alertConfig.has_key("remindName")):
                indexStatus = alertConfig["indexStatus"]
                remindName = alertConfig["remindName"]
                logger.info("===开始执行告警===告警名称：" + remindName + "===");
                # 调度启动后，第一次执行告警规则（达到阈值则可以告警）
                # 第一次告警后，需要考虑<禁止重复告警时间>是否达到
                flag = isIndexExists(indexStatus + "-*");
                Map<String, Object> srStatus = null;
                Integer hitsStatus = 0;
                if (flag == true) {
                    # 从status中查询一条指定规则名称下的最新的记录
                    srStatus = searchStatus(indexStatus, remindName);
                    if (srStatus.has_key("hitsTotal")) {
                        hitsStatus = (Integer) srStatus.get("hitsTotal");
                    }
                }

                # hits.total为0时，插入运行本规则的第一条记录（不管是否达到阈值）
                if (hitsStatus == 0 || !flag) {
                    logger.info("===首次执行告警方法===告警名称：" + remindName + "===");
                    # 返回查询异常所包含的信息（命中条数，耗时）
                    
                    index = "1012-knowledge_training_normal-*"
                    es_type = "knowledge_training_normal"
                    beginTime = "2018-01-26T00:00:00.000+08:00"
                    endTime = "2018-01-27T00:00:00.000+08:00"
                    size = 10000
                    tag = "isException"
                    logmsg = "logBody"
                    source_include = [tag, logmsg]                    
                    es_util = Elasticsearch_Util()
                    train_data_normal = getMoreDataFromES(es_util, index=index, doc_type=es_type,
                                               beginTime=beginTime, endTime=endTime,
                                               size=size,
                                               source_include=source_include)
                    #预测数据
                    for data in train_data_normal:
                        print data
                        error_predict(train_data_normal,logmsg)
                    
                    
                    
                    
                    
                    Map<String, Object> eventNumMonitorMap = searchRule.eventNumMonitor(alertConfig);
                    if (eventNumMonitorMap.containsKey("starttime") && eventNumMonitorMap.containsKey("endtime")
                            && eventNumMonitorMap.containsKey("hits") && eventNumMonitorMap.containsKey("time_taken")) {
                        Object starttime = eventNumMonitorMap.get("starttime");
                        Object endtime = eventNumMonitorMap.get("endtime");
                        Long hits = (Long) eventNumMonitorMap.get("hits");
                        Object time_taken = eventNumMonitorMap.get("time_taken");
                        alertConfig.put("starttime", starttime);
                        alertConfig.put("endtime", endtime);
                        alertConfig.put("hits", hits);
                        alertConfig.put("time_taken", time_taken);

                        hitstotal = hits;
                        search_response = eventNumMonitorMap.get("search_response");

                        flagalert = thresholdValueComparision(alertConfig);
                        
                    }
                } else {
                    String timestampStatus = (String) srStatus.get("timestamp");
                    String starttime = (String) srStatus.get("endtime");
                    alertConfig.put("starttime", starttime);
                    String timeRange = (String) alertConfig.get("timeRange");
                    // 超过或等于调度时间范围
                    if (ComparisonUtil.timeComparision(timestampStatus, timeRange)) {
                        // 返回查询异常所包含的信息（命中条数，耗时）
                        Map<String, Object> eventNumMonitorMap = searchRule.eventNumMonitor(alertConfig);
                        if (eventNumMonitorMap.containsKey("starttime") && eventNumMonitorMap.containsKey("endtime")
                                && eventNumMonitorMap.containsKey("hits") && eventNumMonitorMap.containsKey("time_taken")) {
                            Object endtime = eventNumMonitorMap.get("endtime");
                            Long hits = (Long) eventNumMonitorMap.get("hits");
                            Object time_taken = eventNumMonitorMap.get("time_taken");
                            alertConfig.put("endtime", endtime);
                            alertConfig.put("hits", hits);
                            alertConfig.put("time_taken", time_taken);

                            hitstotal = hits;
                            search_response = eventNumMonitorMap.get("search_response");

                            flagalert = thresholdValueComparision(alertConfig);
                            

                        }
                    } else {
                        // 小于调度时间范围，不执行指定索引下的搜索
                        logger.info("===小于调度时间范围，不执行指定索引下的搜索===告警名称：" + remindName + "===");
                    }
                }

                if (flagalert) {
                    StringBuffer value = new StringBuffer();
                    if (alertConfig.containsKey("index") && alertConfig.containsKey("remindName")
                            && alertConfig.containsKey("fieldValue")) {
                        String index = alertConfig.get("index").toString();
                        String fieldValue = alertConfig.get("fieldValue").toString();
                        String thresholdValue = alertConfig.get("thresholdValue").toString();
                        value.append("监控名称:" + remindName);
                        value.append(";模型:" + index);
                        value.append(";时间范围:" + searchTimeRange);
                        value.append(";搜索条件:" + fieldValue);
                        value.append(";查询结果:" + hitsStr);
                        value.append(";阈值:" + thresholdValue);
                    }
                    value.append(";日志内容:" + getSearchResponseResult(search_response.toString()) + ";");
                    // if(hitstotal>5){
                    // value.append("日志内容过多,请自行查询！");
                    // }
                    alertConfig.put("remindMessage", value.toString());
                    insertElastalert(alertConfig);
                }
            }

	}



In [ ]:
public Map<String, Object> eventNumMonitor(Map<String, Object> searchCondition) throws Exception {
		Map<String, Object> map = new HashMap<String, Object>();
		if (searchCondition.containsKey("index") && searchCondition.containsKey("timeFieldName")
				&& searchCondition.containsKey("fieldValue") && searchCondition.containsKey("timeRange")) {
			String index = (String) searchCondition.get("index");
			String timeFieldName = (String) searchCondition.get("timeFieldName");
			String fieldValue = (String) searchCondition.get("fieldValue");
			String timeRange = "-" + (String) searchCondition.get("timeRange");
			// String timeRange = "-"+"40d";

			String[] split = index.split("-");
			StringBuffer sb = new StringBuffer();

			if (split.length > 2) {
				int i;
				for (i = 1; i < split.length - 2; i++) {
					sb.append(split[i] + "-");
				}
				sb.append(split[i]);
				String type = sb.toString();

				String nowtime = DateUtil.getStandardCurrentTime();
				String starttime = null;
				if (searchCondition.containsKey("starttime")) {
					starttime = searchCondition.get("starttime").toString();
				} else {
					long time = DateUtil.getTime(nowtime, "yyyy-MM-dd'T'HH:mm:ss.SSS'+08:00'");
					long timeRangeMillionsecond = DateUtil.getMillisecondValue(timeRange);
					starttime = DateUtil.format(time + timeRangeMillionsecond, "yyyy-MM-dd'T'HH:mm:ss.SSS'+08:00'");
				}
				String endtime = nowtime;
				StringBuffer sbtimeRange = new StringBuffer("[").append(starttime).append(" TO ").append(endtime)
						.append("}");

				String searchJson = timeFieldName + ":" + sbtimeRange.toString() + " AND _all:" + fieldValue;

				SearchResponse sr = client.search(new String[] { index }, type, searchJson, 1, 10000, null, null);
				Long hits = sr.getHits().getTotalHits();
				String took = sr.getTookInMillis() + "";
				map.put("starttime", starttime);
				map.put("endtime", nowtime);
				map.put("hits", hits);
				map.put("time_taken", took);
				map.put("search_response", sr.toString());
				logger.info("===" + index + "：包含" + fieldValue + "的条数是" + hits + "===");
			}
		}
		return map;
	}


In [ ]:
public void eventNumMonitorAlertRule(String alertJson) throws Exception {
		Map<String, Object> alertConfig = JsonToMapUtil.fieldValueJsonToMap(alertJson);
		alertCommon(alertConfig);
	}

alertConfig = = {'Name': 'Zara', 'Age': 7, 'Class': 'First'};

In [ ]:
package com.link.log.alert;

import java.util.HashMap;
import java.util.Map;

import net.sf.json.JSONArray;
import net.sf.json.JSONObject;

import org.elasticsearch.action.search.SearchResponse;
import org.slf4j.Logger;
import org.slf4j.LoggerFactory;
import org.springframework.beans.factory.annotation.Autowired;

import com.link.elasticsearth.client.CommonESJavaClient;
import com.link.log.engine.ElasticSearchEngine;
import com.link.log.util.ComparisonUtil;
import com.link.uaap.util.DateUtil;

class BaseRule {
	private static final Logger logger = LoggerFactory.getLogger(BaseRule.class);

	@Autowired
	private CommonESJavaClient client;

	@Autowired
	private ElasticSearchEngine elasticSearchEngine;

	/**
	 * 判断是否达到阈值
	 * 
	 * @param map
	 * @return
	 */
thresholdValueComparision(Map<String, Object> map) {
	boolean flag = false;
	if (map.containsKey("remindName") && map.containsKey("operator") && map.containsKey("thresholdValue")
			&& map.containsKey("hits")) {
		String remindName = map.get("remindName").toString();
		String operator = map.get("operator").toString();
		Object thresholdValue = map.get("thresholdValue");
		Object hits = map.get("hits");
		if (ComparisonUtil.valueComparision(operator, thresholdValue, hits)) {
			insertStatus(map);
			flag = alert(map);
		} else {
			insertStatus(map);
			logger.info("===没有触发阈值，记录日志，不告警===告警名称：" + remindName + "===");
		}
	}
	return flag;
}

	/**
	 * 告警
	 * 
	 * @param map
	 * @return
	 */
alert(Map<String, Object> map) {
	boolean flag = false;
	if (map.containsKey("indexElastalert") && map.containsKey("remindName")) {
		String indexElastalert = (String) map.get("indexElastalert");
		String remindName = (String) map.get("remindName");
		boolean isElastalertExit = isIndexExists(indexElastalert + "-*");
		Map<String, Object> srElastalert = null;
		Integer hitsElastalert = 0;
		if (isElastalertExit == true) {
			// 从Elastalert中查询一条指定规则名称下的最新的记录
			srElastalert = searchElastalert(indexElastalert, remindName);
			hitsElastalert = (Integer) srElastalert.get("hitsTotal");
		}
		if (hitsElastalert == 0 || !isElastalertExit) {
			flag = true;
			logger.info("===第一次调用告警方式===告警名称：" + remindName + "===");
		} else {
			String timestampElastalert = (String) srElastalert.get("timestamp");
			map.put("timestampElastalert", timestampElastalert);
			// 超过或等于禁止重复警告时间
			if (map.containsKey("realertTimeRange")) {
				String realertTimeRange = (String) map.get("realertTimeRange");
				if (ComparisonUtil.timeComparision(timestampElastalert, realertTimeRange)) {
					flag = true;
					logger.info("===超过或等于禁止重复警告时间，执行告警===告警名称：" + remindName + "===");
				} else {
					// 小于禁止重复告警时间，不执行告警
					logger.info("===小于禁止重复告警时间，不执行告警===告警名称：" + remindName + "===");
				}
			}

		}
	}
	return flag;
}

	/**
	 * 判断索引是否存在
	 * 
	 * @param index
	 * @return
	 */
	public boolean isIndexExists(String index) {
		if (index != null) {
			return client.isIndexExists(index);
		}
		return false;
	}

	/**
	 * 从status中查询一条指定规则名称下的最新的记录
	 * 
	 * @param index
	 * @param remindName
	 * @return
	 * @throws Exception
	 */
	public Map<String, Object> searchStatus(String index, String remindName) throws Exception {
		Map<String, Object> map = new HashMap<String, Object>();
		if (index != null && index != "" && remindName != null && remindName != "") {
			String type = indexToType(index);

			// String searchJson = " rule_name: " + remindName;
			// SearchResponse sr = client.search(new String[] { index + "-*" },
			// type, searchJson, 1, 1, null);

			SearchResponse sr = client.indicatorSearch(index + "-*", type, new String[] { "rule_name=" + remindName },
					1, 1);
			JSONObject object = JSONObject.fromObject(sr.toString());
			JSONObject object2 = (JSONObject) object.get("hits");
			// 查询出的记录条数
			Integer hitsTotal = (Integer) object2.get("total");
			map.put("hitsTotal", hitsTotal);
			
			if (hitsTotal != 0) {
				// hits.total为0时，插入运行本规则的第一条记录
				JSONArray array1 = (JSONArray) object2.get("hits");
				JSONObject object3 = (JSONObject) array1.get(0);
				JSONObject object4 = (JSONObject) object3.get("_source");
				
				if (object4.containsKey("@timestamp")) {
					String timestamp = object4.getString("@timestamp");
					map.put("timestamp", timestamp);
				}
				if (object4.containsKey("endtime")) {
					String endtime = object4.getString("endtime");
					map.put("endtime", endtime);
				}
				if (object4.containsKey("hits")) {
					String hits = object4.getString("hits");
					map.put("hits", hits);
				}
			}
			logger.info("===搜索查询记录,告警名称：" + remindName + ",搜索结果：" + sr.toString() + "===");
			return map;
		}
		return map;
	}

	/**
	 * 向status中插入记录
	 * 
	 * @param index
	 * @param elastalertStatus
	 */
	public void insertStatus(Map<String, Object> map) {
		if (map.containsKey("indexStatus")) {
			String index = map.get("indexStatus").toString() + "-" + DateUtil.getDate();
			String[] split = index.split("-");
			if (split.length > 2) {
				StringBuffer sb = new StringBuffer();
				int i;
				for (i = 1; i < split.length - 2; i++) {
					sb.append(split[i] + "-");
				}
				sb.append(split[i]);
				String type = sb.toString();
				if (map.containsKey("remindName") && map.containsKey("starttime") && map.containsKey("endtime")
						&& map.containsKey("time_taken")) {
					String remindName = map.get("remindName").toString();

					// 搜索记录中需要记录的内容
					String starttime = map.get("starttime").toString();
					String endtime = map.get("endtime").toString();

					String hits = "";
					if (map.containsKey("hits")) {
						hits = map.get("hits").toString();
					}

					String matches = "";
					if (map.containsKey("thresholdValue")) {
						matches = map.get("thresholdValue").toString();
					}

					String time_taken = map.get("time_taken").toString();

					String status_info = "";
					if (map.containsKey("status_info")) {
						status_info = map.get("status_info").toString();
					}
					JSONObject bodyJson = new JSONObject();
					bodyJson.put("@timestamp", DateUtil.getStandardCurrentTime());
					bodyJson.put("rule_name", remindName);
					bodyJson.put("starttime", starttime);
					bodyJson.put("endtime", endtime);
					bodyJson.put("hits", hits);
					bodyJson.put("matches", matches);
					bodyJson.put("time_taken", time_taken);
					bodyJson.put("status_info", status_info);

					if (map.containsKey("schedule_batchid")) {
						String schedule_batchid = map.get("schedule_batchid").toString();
						bodyJson.put("schedule_batchid", schedule_batchid);
					}

					elasticSearchEngine.insertESData("1", type, "common", bodyJson.toString());
					logger.info("===记录搜索日志===[提醒名称：" + remindName + ",starttime:" + starttime + ",endtime:" + endtime
							+ ",hits:" + hits.toString() + ",matches:" + matches + ",time_taken:" + time_taken
							+ ",status_info:" + status_info + "]===");
				}
			}
		}

	}

	/**
	 * 从Elastalert中查询一条指定规则名称下的最新的记录
	 * 
	 * @param index
	 * @param remindName
	 * @return
	 * @throws Exception
	 */
	public Map<String, Object> searchElastalert(String index, String remindName) {
		Map<String, Object> map = new HashMap<String, Object>();
		if (index != null && index != "" && remindName != null && remindName != "") {
			String type = indexToType(index);

			String searchJson = " rule_name: " + remindName;
			SearchResponse sr = null;
			try {
				sr = client.search(new String[] { index + "-*" }, type, searchJson, 1, 1, null, null);
			} catch (Exception e) {
				logger.error(e.getMessage());
			}

			JSONObject object = JSONObject.fromObject(sr.toString());
			JSONObject object2 = (JSONObject) object.get("hits");
			// 查询出的记录条数
			Integer hitsTotal = (Integer) object2.get("total");
			map.put("hitsTotal", hitsTotal);
			if (hitsTotal != 0) {
				// hits.total为0时，插入运行本规则的第一条记录
				JSONArray array1 = (JSONArray) object2.get("hits");
				JSONObject object3 = (JSONObject) array1.get(0);
				JSONObject object4 = (JSONObject) object3.get("_source");
				if (object4.containsKey("@timestamp")) {
					String timestamp = object4.getString("@timestamp");
					map.put("timestamp", timestamp);
				}
			}
			logger.info("===搜索告警记录,告警名称：" + remindName + ",搜索结果：" + sr.toString() + "===");
		}
		return map;
	}

	/**
	 * 向Elastalert中插入记录
	 * 
	 * @param index
	 * @param elastalert
	 */
	public void insertElastalert(Map<String, Object> map) {

		if (map.containsKey("indexElastalert")) {
			String index = map.get("indexElastalert").toString() + "-" + DateUtil.getDate();
			String[] split = index.split("-");
			if (split.length > 2) {
				StringBuffer sb = new StringBuffer();
				int i;
				for (i = 1; i < split.length - 2; i++) {
					sb.append(split[i] + "-");
				}
				sb.append(split[i]);
				String type = sb.toString();

				if (map.containsKey("remindName") && map.containsKey("remindMessage") && map.containsKey("remindWay")
						&& map.containsKey("remindPerson") && map.containsKey("remindEmail")
						&& map.containsKey("index")) {
					String remindName = map.get("remindName").toString();
					String remindMessage = map.get("remindMessage").toString();
					String remindWay = map.get("remindWay").toString();

					String searchIndex = map.get("index").toString();

					// 提醒人与提醒人邮箱是一一对应的
					String remindPersonStr = map.get("remindPerson").toString();// 提醒人a,b,c,d
					String remindEmailStr = map.get("remindEmail").toString();// 提醒人邮箱d,e,f,g

					String remindPerson[] = remindPersonStr.split(",");
					String remindEmail[] = remindEmailStr.split(",");

					if (remindPerson.length > 0 && remindEmail.length > 0
							&& remindPerson.length == remindEmail.length) {
						for (int j = 0; j < remindPerson.length; j++) {
							JSONObject bodyJson = new JSONObject();
							String timestamp = DateUtil.getStandardCurrentTime();
							bodyJson.put("@timestamp", timestamp);// 创建时间
							bodyJson.put("remindName", remindName);// 提醒名称
							bodyJson.put("remindType", "alert");// 提醒类型（指标提醒idct，提醒提醒alert）
							bodyJson.put("remindStatus", "待发送");// 提醒状态（3种：待发送，发送成功，发送失败）
							bodyJson.put("remindMessage", remindMessage);// 提醒内容
							bodyJson.put("remindWay", remindWay);// 提醒方式
							bodyJson.put("remindPerson", remindPerson[j]);// 提醒人
							bodyJson.put("remindEmail", remindEmail[j]);// 提醒人邮箱

							// 2017-11-29T15:38:04.482+08:00
							String timestamps = timestamp.substring(0, timestamp.indexOf("+")).replaceAll("-", "")
									.replaceAll(":", "").replaceAll("T", "").replaceAll("\\.", "");
							String msgId = remindName + "_" + searchIndex + "_" + timestamps;

							bodyJson.put("msgId", msgId);// 消息主键
							bodyJson.put("searchIndex", searchIndex);// 模型名称

							if (map.containsKey("schedule_batchid")) {
								String schedule_batchid = map.get("schedule_batchid").toString();
								bodyJson.put("schedule_batchid", schedule_batchid);
							}

							if (map.containsKey("fieldValue")) {
								String searchCondition = map.get("fieldValue").toString();
								bodyJson.put("searchCondition", searchCondition);
							}

							elasticSearchEngine.insertESData("1", type, "common", bodyJson.toString());
							logger.info("记录告警执行记录,消息提醒状态【待发送】,提醒类型【alert】===[提醒名称：" + remindName + ",提醒人:"
									+ remindPerson[j] + ",提醒人邮箱:" + remindEmail[j] + ",提醒内容:" + remindMessage + "]===");
						}
					}
				}
			}
		}
	}

	/**
	 * 从index中获得type
	 * 
	 * @param index
	 * @return
	 */
	public String indexToType(String index) {
		if (index != null && index.length() > 0) {
			String[] split = index.split("-");
			StringBuffer sb = new StringBuffer();
			int i;
			for (i = 1; i < split.length - 1; i++) {
				sb.append(split[i] + "-");
			}
			sb.append(split[i]);
			return sb.toString();
		}
		return "index为空";
	}

}
